In [2]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix

model = tf.keras.models.load_model(
    "gender_age_hair_model.h5",
    custom_objects={
        'mse': tf.keras.losses.MeanSquaredError(),
        'binary_crossentropy': tf.keras.losses.BinaryCrossentropy(),
        'mae': tf.keras.metrics.MeanAbsoluteError(),
        'accuracy': tf.keras.metrics.BinaryAccuracy()
    }
)

def load_test_data(dataset_path, img_size=(128, 128)):
    images, ages, genders = [], [], []
    
    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg"):
            parts = filename.split("_")
            age = int(parts[0])
            gender = int(parts[1])
            img = Image.open(os.path.join(dataset_path, filename)).convert("RGB")
            img = img.resize(img_size)
            img = np.array(img) / 255.0
            images.append(img)
            ages.append(age)
            genders.append(gender)
    
    images = np.array(images)
    ages = np.array(ages)
    genders = np.array(genders)
    
    hair_length = []
    for age, gender in zip(ages, genders):
        if 20 <= age <= 30:
            hair_length.append(gender)
        else:
            hair_length.append(1 - gender)
    hair_length = np.array(hair_length)
    
    return images, ages, genders, hair_length

test_dataset_path = "UTKFace"
X_test, y_age_test, y_gender_test, y_hair_test = load_test_data(test_dataset_path)

eval_results = model.evaluate(
    X_test,
    {'age': y_age_test, 'gender': y_gender_test, 'hair_length': y_hair_test},
    verbose=1
)

print("\nEvaluation Results:")
print(f"Total Loss: {eval_results[0]:.4f}")
print(f"Age Loss: {eval_results[1]:.4f} | Age MAE: {eval_results[4]:.4f}")
print(f"Gender Loss: {eval_results[2]:.4f} | Gender Accuracy: {eval_results[5]:.4f}")
print(f"Hair Length Loss: {eval_results[3]:.4f} | Hair Length Accuracy: {eval_results[6]:.4f}")

_, gender_preds, _ = model.predict(X_test)
gender_preds_bin = (gender_preds > 0.5).astype(int)
print("\nGender Prediction Classification Report:")
print(classification_report(y_gender_test, gender_preds_bin))
print("Confusion Matrix:")
print(confusion_matrix(y_gender_test, gender_preds_bin))


741/741 ━━━━━━━━━━━━━━━━━━━━ 24s 29ms/step - age_loss: 27.7493 - age_mean_absolute_error: 3.2733 - gender_accuracy: 0.9133 - gender_loss: 0.2204 - hair_length_accuracy: 0.7397 - hair_length_loss: 0.5412 - loss: 28.5109

Evaluation Results:
Total Loss: 38.7545
Age Loss: 38.1393 | Age MAE: 3.8095
Gender Loss: 0.1713 | Gender Accuracy: 0.9355
Hair Length Loss: 0.4418 | Hair Length Accuracy: 0.7960
741/741 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step

Gender Prediction Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     12391
           1       0.94      0.92      0.93     11317

    accuracy                           0.94     23708
   macro avg       0.94      0.93      0.94     23708
weighted avg       0.94      0.94      0.94     23708

Confusion Matrix:
[[11764   627]
 [  902 10415]]
